In [11]:
%%writefile ../scripts/plot_hicnv_bedgraph.py
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import argparse
import seaborn as sns
import pytls

# ################# Parsing the command line arguments ##################
# Current coord-type is only handling genomic data since the resolution we
# are using is fragment based and dynamic.
# I am thinking of implementing a bin based approach where you just use the
# index associated with a fragment but that is currently not implemented.
parser = argparse.ArgumentParser()
parser.add_argument('--bedgraph', type=str)
parser.add_argument('--outfn', type=str)
parser.add_argument('--coord-type', default='bin', choices=['bin', 'genomic'])
parser.add_argument('--max-cn', default=6)
#params = parser.parse_args()

params = parser.parse_known_args()[0]

params.bedgraph = '../../results/main/22Rv1/hicnv/run/22Rv1_SRR7760384_hicnv/CNV_Estimation/22Rv1.SRR7760384.cnv.bedGraph'
params.outfn = 'comprehensive_cnv_bedgraph.png'
params.coord_type = 'genomic'

# ################# Loading and parsing the data ##################
print('Loading and parsing the data')
hicnv = pd.read_table(params.bedgraph, header=None)
hicnv.columns = ['chr', 'start', 'end', 'counts', 'cn', 'cn_cat']
hicnv.loc[:, 'chr'] = ['chr{}'.format(x) for x in
                       hicnv['chr'].apply(pytls.chrName_to_chrNum)]

# group the regionss by their chromosome
hicnv_grps = hicnv.groupby('chr')
chr_dict = [(int(x.replace('chr', '')), x) for x in hicnv_grps.groups.keys()]
chr_dict = sorted(chr_dict, key=lambda x: x[0])

# ################# Plotting all chromosomes on a single image ################
# making the figure + axes and unravelling axes for plotting
fig, axes = plt.subplots(figsize=(8, 11),
                         nrows=6,
                         ncols=4,
                         gridspec_kw={'hspace': 0.8, 'wspace': 0.5})
axes = np.ravel(axes)

# plotting each chromosome onto it's own axis
for chrom_num, chrom in chr_dict:
    
    print('Plotting: chr', chrom_num)

    # get the ax
    ax = axes[chrom_num - 1]

    # extract the data for the current chrom
    hicnv_grp_df = hicnv_grps.get_group(chrom)
    
    # extract the x and corresponding y points
    x_vals = []
    y_vals = []
    for (start, end, cn) in hicnv_grp_df[['start', 'end', 'cn']].values:

        # add the start 
        x_vals.append(start)
        y_vals.append(cn)

        # add the end
        x_vals.append(end)
        y_vals.append(cn)
    
    # plot a line plot
    ax.plot(x_vals, y_vals, color='blue')

    # set the axis labels
    ax.set_title('{}'.format(chrom))
    if chrom_num > 20:
        ax.set_xlabel('Bin')

    if chrom_num % 4 == 1:
        ax.set_ylabel('CN')

    # set the ytick labels
    ax.set_ylim(-0.2, max_cn)
    ax.yaxis.set_ticks(range(0, params.max_cn, 2))
    ax.yaxis.set_ticks(range(1, params.max_cn, 2), minor=True)
    
    # set a grid on the y-axis for easier visualization
    ax.grid(which='both', axis='y')

    # set the xticks at every 10th quartile
    #chrom_len = int(hicnv_grp_df['end'].max()) * 2
    #interval = int(chrom_len / 10)
    #r = range(0, chrom_len + 1, interval)
    #ax.set_xticks(r, minor=False)

axes[23].set_visible(False)
fn = os.path.join(params.outfn)
fig.savefig(fn, dpi=200)

Writing ../scripts/plot_hicnv_bedgraph.py


In [12]:
hicnv_grp_df

,chr,start,end,counts,cn,cn_cat
512405,chr23,0,2780881,12.090,0.325,D
512406,chr23,2780881,2787682,12.090,0.325,D
512407,chr23,2787682,2792397,12.090,0.325,D
512408,chr23,2792397,2796988,12.090,0.325,D
512409,chr23,2796988,2802257,12.090,0.325,D
...,...,...,...,...,...,...
540761,chr23,155676142,155683767,13.097,0.352,D
540762,chr23,155683767,155687501,13.097,0.352,D
540763,chr23,155687501,155692925,13.097,0.352,D
540764,chr23,155692925,155699776,13.097,0.352,D
